In [1]:
from utils.setup_utils import setup
setup(globals())

W, B = load_model()
x, y = load_data()

x = (x - 0.1307) / 0.3081
x = x.reshape(-1, 28 * 28)

def ginit(shape):
    return np.zeros(shape, dtype=gp.Var)

In [55]:
true_label = y[0]
fake_label = 1
image = x[0]

m = gp.Model()

gurobi_x = [ginit(x.shape[1]), ginit(B[0].shape), ginit(B[1].shape), ginit(B[2].shape)]
gurobi_z = [None, ginit(B[0].shape), ginit(B[1].shape), ginit(B[2].shape)]


##### Variables #####

for j in range(x.shape[1]):
    gurobi_x[0][j] = m.addVar(vtype='C', name=f'x_0_{j}') 
#                               lb=(-1 - 0.1307) / 0.3081, ub=(1 - 0.1307) / 0.3081)

for layer_index in range(3):
    w = W[layer_index]
    b = B[layer_index]
    
    n_k_prev, n_k = w.shape
    
    for j in range(n_k):
        gurobi_x[layer_index + 1][j] = m.addVar(vtype='C', name=f'x_{layer_index}_{j}')
        gurobi_z[layer_index + 1][j] = m.addVar(vtype='C', name=f'z_{layer_index}_{j}')
        
##### Constraints #####

for layer_index in range(3):
    w = W[layer_index]
    b = B[layer_index]
    
    n_k_prev, n_k = w.shape
    
    for j in range(n_k):
        
        # feed forward
        _ = m.addConstr(gp.quicksum(w[i][j] * gurobi_x[layer_index][i] 
                                 for i in range(n_k_prev)) \
                        + b[j] == \
                    gurobi_z[layer_index + 1][j]
                   )
        
        # ReLU activation
        
#         _ = m.addConstr((gurobi_x[layer_index + 1][j] != gurobi_z[layer_index + 1][j]) >> 
#                         (gurobi_z[layer_index + 1][j] <= 0))
        
#         _ = m.addConstr((gurobi_x[layer_index + 1][j] != 0) >>
#                         (gurobi_z[layer_index + 1][j] >= 0))
        
#         _ = m.addConstr((gurobi_x[layer_index + 1][j] == 0) or \
#                         (gurobi_x[layer_index + 1][j] == gurobi_z[layer_index + 1][j]))
        
#         _ = m.addGenConstrMax(gurobi_x[layer_index + 1][j], 
#                               [gurobi_z[layer_index + 1][j]], 
#                               0)
        
        _ = m.addConstr(gurobi_x[layer_index + 1][j] == \
                        gp.max_([gurobi_z[layer_index + 1][j], 0]))
        
        # https://support.gurobi.com/hc/en-us/community/posts/360077951791-if-statement-in-constraint-

# Lead the model to a false prediction.
_ = m.addConstr(gurobi_x[-1][fake_label] == gp.max_(gurobi_x[-1][i] for i in range(10)))

##### Objective #####

delta = ginit(image.shape[0])

for j in range(image.shape[0]):
    delta[j] = m.addVar(vtype='C', name=f'delta_{j}')
    _ = m.addConstr((gurobi_x[0][j] - image[j]) == delta[j])

m.setObjective(gp.quicksum(delta[i] ** 2 for i in range(image.shape[0])), gp.GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1306 rows, 2612 columns and 270890 nonzeros
Model fingerprint: 0xc32a050e
Model has 784 quadratic objective terms
Model has 523 general constraints
Variable types: 2612 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-08, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-05, 3e+00]
Presolve removed 262 rows and 793 columns
Presolve time: 0.34s
Presolved: 1044 rows, 1819 columns, 270366 nonzeros
Presolved model has 784 quadratic objective terms
Variable types: 1819 continuous, 0 integer (0 binary)
Presolve removed 513 rows and 513 columns
Presolve time: 0.48s
Presolved: 531 rows, 1306 columns, 269340 nonzeros
Presolved model has 784 quadratic objective terms
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' 

In [43]:
constructed = np.array([gurobi_x[0][j].X for j in range(image.shape[0])]).reshape(28, 28)

In [57]:
from PIL import Image

t = np.uint8((constructed * 255).clip(0, 255))
Image.fromarray(t)

In [70]:
x1 = image.reshape(-1, 28 * 28) @ W[0] + B[0]
y1 = relu(x1)

x2 = y1 @ W[1] + B[1]
y2 = relu(x2)

x3 = y2 @ W[2] + B[2]

x3.argmax(axis=1)
x3

array([7], dtype=int64)

array([[ -1.70159475,  -1.19535526,   0.50834242,   3.22029661,
         -4.15990621,  -2.5535982 , -11.01674886,  16.0265664 ,
         -3.08458593,   4.16988858]])

In [68]:
x1 = constructed.reshape(-1, 28 * 28) @ W[0] + B[0]
y1 = relu(x1)

x2 = y1 @ W[1] + B[1]
y2 = relu(x2)

x3 = y2 @ W[2] + B[2]

x3.argmax(axis=1)
x3

array([1], dtype=int64)

array([[ 6.83304593e-11,  1.37262031e-01,  2.42658671e-12,
         2.76063061e-11,  6.10661521e-12,  1.37262031e-01,
        -4.77989870e-12,  1.37262031e-01,  1.53511648e-11,
        -1.84729870e-11]])